In [ ]:
%sql -- Extração enriquecida de ordens

WITH ordens_filtradas AS (
    SELECT aufk.AUFNR, aufk.MANDT
    FROM poseidon_uc.saps4hanarawview.aufk AS aufk
    WHERE aufk.AUART IN ('GIA','GAA','GRA') 
    -- AND aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
    AND aufk.BUKRS IN ('GHIN')
    AND aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )
    LIMIT 10  -- Processar em lotes
)
SELECT
    aufk.AUFNR AS ordem,
    aufk.AUART AS tipo_ordem,
    aufk.BUKRS AS empresa,
    aufk.KTEXT AS texto_breve,
    aufk.ERDAT AS data_criacao,
    aufk.WERKS AS centro_trabalho,
    
    -- Apenas campos essenciais
    afko.GSTRP AS data_inicio_prog,
    afko.GLTRP AS data_fim_prog,
    afko.GAMNG AS quantidade_ordem,
    
    afih.EQUNR AS equipamento,
    afih.PRIOK AS prioridade,
    afih.QMNUM AS notificacao,
    
    afpo.MATNR AS material,
    afpo.PSMNG AS quantidade_material,
    
    qmel.QMART AS tipo_defeito,
    qmel.QMGRP AS grupo_qualidade

FROM ordens_filtradas of
INNER JOIN poseidon_uc.saps4hanarawview.aufk AS aufk
    ON aufk.AUFNR = of.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.afko AS afko
    ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.afih AS afih
    ON afih.MANDT = aufk.MANDT AND afih.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.afpo AS afpo
    ON afpo.MANDT = aufk.MANDT AND afpo.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.qmel AS qmel
    ON qmel.AUFNR = aufk.AUFNR

ORDER BY aufk.AUFNR